SQL Homework - Employee Database: A Mystery in Two Parts: 
    - Objective: generate a visualization of the data fromm the Employee_db analysis
    - Requirements: Import the SQL database into Pandas.
                    Create a histogram to visualize the most common salary ranges for employees.
                    Create a bar chart of average salary by title.    

In [ ]:
# Dependencies and Setup
%matplotlib inline
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import pandas as pd
from config import jon
from config import Employees_db

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost:5432/Employee_db')
connection = engine.connect()